In [1]:
from nsepython import *

import pandas as pd
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Create Dataset

In [2]:
%%capture
fno_symbols = fnolist()
fno_stocks = [symbol for symbol in fno_symbols if symbol not in ['NIFTY', 'NIFTYIT', 'BANKNIFTY']]

In [5]:
fno_symbols = fnolist()

In [3]:
stock_data = pd.read_csv('stock_data.csv')
stock_data.set_index('SYMBOL', inplace=True)
fno_stocks = stock_data[stock_data.index.isin(fno_symbols)]

In [ ]:
fno_stocks = fno_stocks.sort_values(by=['INDUSTRY', 'Mar Cap Rs.Cr.'], ascending=[True, False])

### Plot Stock Charts

In [ ]:
def download_stock_data(symbol):
    stock_data = yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
    stock_data.columns = [col[0] for col in stock_data.columns]
    return stock_data
    

def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data

def plot_stock_data(symbol, data):
    # Filter data to only include the last 360 days
    plot_data = data[data.index > (pd.Timestamp.now() - pd.DateOffset(days=270))]

    # Setup subplot layout
    figure = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    
    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'), row=1, col=1)
    
    # Plot moving averages
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'), row=1, col=1)

    # Plot volume with color-coded bars (green for positive, red for negative days)
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True, row=1, col=1)

    # Update axes
    figure.update_yaxes(range=[plot_data['Close'].min()*0.8, plot_data['Close'].max()*1.2], row=1, col=1)
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # Hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True, row=1, col=1)
    figure.update_yaxes(visible=False, secondary_y=True, row=1, col=1)

    return figure


def process_stocks(all_symbols):
    num_processed = 0
    file_count = 1

    while num_processed < len(all_symbols):
        # Create a new HTML file for each batch of 120 symbols
        file_name = f'fno_stocks_{file_count}.html'
        with open(file_name, 'w') as figure_html:
            for symbol in all_symbols[num_processed:num_processed+120]:
                try:
                    # Download stock data
                    data = download_stock_data(symbol)
                    # Calculate technical indicators
                    data = calculate_technical_indicators(data)
                    # Generate the plot
                    figure = plot_stock_data(symbol, data)
                    # Append the figure's HTML to the file
                    figure_html.write(figure.to_html(full_html=False))
                except Exception as e:
                    print(f"Error processing symbol {symbol}: {e}")
        print(f"Completed file: {file_name}")
        # Update the count of processed symbols
        num_processed += 120
        file_count += 1
    print(f"Processing completed. Total files created: {file_count - 1}")


if __name__ == "__main__":
    process_stocks(fno_symbols)
    print('Execution Completed!!!!')

In [ ]:
# def download_stock_data(symbol):
#     return yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
#                        end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
# 
# def calculate_technical_indicators(data):
#     data['5DMA'] = data['Close'].rolling(window=5).mean()
#     data['20DMA'] = data['Close'].rolling(window=20).mean()
#     data['50DMA'] = data['Close'].rolling(window=50).mean()
#     data['100DMA'] = data['Close'].rolling(window=100).mean()
#     data['diff'] = data['Close'] - data['Open']
#     data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
#     return data
# 
# def plot_stock_data(symbol, data):
#     # Prepare plot data
#     plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=180))]
# 
#     # Setup subplot layout
#     figure = make_subplots(rows=1, cols=2, column_widths=[0.75, 0.25],
#                            specs=[[{"secondary_y": True}, {"type": "table"}]])
# 
#     # Plot candlestick chart
#     figure.add_trace(go.Candlestick(x=plot_data.index,
#                                     open=plot_data['Open'],
#                                     high=plot_data['High'],
#                                     low=plot_data['Low'],
#                                     close=plot_data['Close'],
#                                     name='Price'), row=1, col=1)
# 
#     # Adjust Y-axis and layout for stock prices
#     figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05], row=1, col=1)
#     figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # hide weekends
#     figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)
# 
#     # Plot moving averages and volume
#     for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
#         figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'), row=1, col=1)
#     figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
#                             marker={'color': plot_data['color']}), secondary_y=True, row=1, col=1)
#     
# 
#     # Hide the secondary Y-axis (volume)
#     figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True, row=1, col=1)
#     figure.update_yaxes(visible=False, secondary_y=True, row=1, col=1)
# 
#     # Plot stock characteristics in a table
#     characteristics = calculate_stock_characteristics(symbol)
#     table_trace = go.Table(
#                 header=dict(values=['Attribute', 'Values'],
#                             fill_color='paleturquoise',
#                             align='left'),
#                 cells=dict(values=[list(characteristics.transpose().index), list(characteristics.transpose().values)],
#                            fill_color='lavender',
#                            align='left'))
#     figure.add_trace(table_trace, row=1, col=2)
#     return figure
# 
# def calculate_stock_characteristics(symbol):
#     stock_row = stock_data.loc[symbol]
#     characteristics = pd.DataFrame({
#                 "Attribute": ["INDUSTRY", "SECTOR", "MARKET_CAP", "FNO", "P/E", "QoQ-SALES", "QoQ-PROFIT", "YoY-SALES", "YoY-PROFIT", "PROMOTER", "DII", "FII", "CHANGE-PROMOTER", "EXP-SALES-GROWTH", "RSI", "MACD", "ROCE"],
#                 "Value": [stock_row['INDUSTRY'], stock_row['SECTOR'], stock_row['Mar Cap Rs.Cr.'], stock_row['FNO'], stock_row['P/E'], stock_row['QoQ Sales %'], stock_row['QoQ Profits %'], stock_row['Qtr Sales Var %'], stock_row['Qtr Profit Var %'], stock_row['Prom. Hold. %'], stock_row['DII Hold %'], stock_row['FII Hold %'], stock_row['Change in Prom Hold %'], stock_row['Exp Qtr Sales Var %'], stock_row['RSI Rs.'], stock_row['MACD Signal Rs.'], stock_row['ROCE']]
#             }).set_index("Attribute").T
#     return characteristics
# 
# 
# def process_stocks(all_symbols):
#     figure_html = open('fno_stocks.html', 'w')
#     for symbol in all_symbols:
#         try:
#             data = download_stock_data(symbol)
#             if not data.empty:
#                 data = calculate_technical_indicators(data)
#                 figure = plot_stock_data(symbol, data)
#                 figure_html.write(figure.to_html(full_html=False))
#         except Exception as e:
#             print(f"Error processing symbol {symbol}: {e.args}")
# 
# if __name__ == "__main__":
#     process_stocks(fno_symbols)
#     print('Execution Completed!!!!')

In [ ]:
def download_stock_data(symbol):
    stock_data = yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)
    stock_data.columns = [col[0] for col in stock_data.columns]
    return stock_data

def calculate_technical_indicators(data):
    data['Close'] = data['Close'] / data['Close'].iloc[0]
    data['Open'] = data['Open'] / data['Open'].iloc[0]
    data['Low'] = data['Low'] / data['Low'].iloc[0]
    data['High'] = data['High'] / data['High'].iloc[0]
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data

def plot_stock_data(data, symbol):
    # Prepare plot data
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=180))]

    # Plot line chart
    line_chart = go.Scatter(x=plot_data.index, y=plot_data['Close'], mode='lines', name=symbol)
    return line_chart

def process_stocks(stock_data):
    fno_stocks_sectors_html = open('fno_stocks_sectors.html', 'w')
    # Group stock symbols by sector
    sector_groups = stock_data.groupby('SECTOR')
    
    # Iterate over sectors
    for sector, stocks in sector_groups:
        # Create a subplot for each sector
        fig = make_subplots(rows=1, cols=1, shared_yaxes=True, 
                            subplot_titles=[f'Sector: {sector}'])
        
        
        # Iterate over symbols in the sector
        for symbol in stocks.index:
            try:
                data = download_stock_data(symbol)
                if not data.empty:
                    data = calculate_technical_indicators(data)
                    line_chart = plot_stock_data(data, symbol)
                    fig.add_trace(line_chart)
            except Exception as e:
                print(f"Error processing symbol {symbol}: {e.args}")
        
        # Update layout
        fig.update_layout(title_text=f'Stocks in Sector: {sector}', xaxis_rangeslider_visible=False)
        
        # Save figure as HTML
        fno_stocks_sectors_html.write(fig.to_html(full_html=False))
        

if __name__ == "__main__":
    process_stocks(fno_stocks)
    print('Execution Completed!!!!')

In [ ]:
monthly_return_volume = pd.DataFrame({'SYMBOL':[], 'SECTOR':[], 'INDUSTRY':[], 'MARKET_CAP':[], 'JAN':[], 'FEB':[], 'MAR':[], 'APR':[],  
                                'MAY':[], 'JUN':[], 'JUL':[], 'AUG':[], 'SEP': [], 'OCT':[], 'NOV':[], 'DEC': []})

for symbol in fno_stocks.index.values:
    try:
        stock = yf.download(symbol + ".NS", start=(pd.to_datetime('today') - pd.DateOffset(days=330)), end=pd.to_datetime('today') + pd.DateOffset(days=1), progress=False)
        
        #start_date = '2022-07-01'
        #end_date = '2023-06-30'
        #stock = yf.download(symbol + ".NS", start=start_date, end=end_date, progress=False)
        
        stock_length = len(stock['Close'])
        
        if stock_length > 200:
            total_volume = stock['Volume'].sum()
            stock['Month'] = stock.index.strftime("%b")
            grouped_by_month = stock.groupby('Month')
            
            monthly_return_data = {'SYMBOL': [symbol], 
                                   'SECTOR': [stock_data.loc[symbol]['SECTOR']],
                                   'INDUSTRY': [stock_data.loc[symbol]['INDUSTRY']],
                                   'MARKET_CAP': [stock_data.loc[symbol]['Mar Cap Rs.Cr.']],
                                   'JAN': 0, 'FEB': 0, 'MAR': 0, 'APR': 0, 'MAY': 0,
                                   'JUN': 0, 'JUL': 0, 'AUG': 0, 'SEP': 0, 'OCT': 0,
                                   'NOV': 0, 'DEC': 0}
            
            monthly_volume_data = {'SYMBOL': [''], 
                                   'SECTOR': [stock_data.loc[symbol]['SECTOR']],
                                   'INDUSTRY': [stock_data.loc[symbol]['INDUSTRY']],
                                   'MARKET_CAP': [stock_data.loc[symbol]['MARKET_CAP']],
                                   'JAN': 0, 'FEB': 0, 'MAR': 0, 'APR': 0, 'MAY': 0,
                                   'JUN': 0, 'JUL': 0, 'AUG': 0, 'SEP': 0, 'OCT': 0,
                                   'NOV': 0, 'DEC': 0}
            
            for month, group in grouped_by_month:
                month_returns = (group.iloc[-1]['Close'] / group.iloc[0]['Close']) - 1
                monthly_return_data[month.upper()] = int(round(month_returns, 2) * 100)
                month_volume = (group['Volume'].sum() * 20) / (total_volume * group.shape[0])
                sign = 1 if month_returns > 0 else -1
                monthly_volume_data[month.upper()] = round((month_volume) * 100 * sign)
            
            
            monthly_return_volume = pd.concat([monthly_return_volume, pd.DataFrame(monthly_return_data)], ignore_index=True)
            monthly_return_volume = pd.concat([monthly_return_volume, pd.DataFrame(monthly_volume_data)], ignore_index=True)
                
    except Exception as e:
        print(f"An error occurred: {symbol}, {str(e)}")

print(f"Execution completed.")

In [ ]:
months = ['OCT', 'SEP', 'AUG', 'JUL', 'JUN', 'MAY', 'APR', 'MAR', 'FEB', 'JAN', 'DEC', 'NOV']
average_volume_by_industry = monthly_return_volume.groupby('INDUSTRY')[months].mean().round(2)

plt.figure(figsize=(20, 15))
sns.heatmap(average_volume_by_industry, cmap='RdYlGn', annot=True, fmt=".2f", vmin=-10, vmax=10)
plt.title('Average Volume by Industry')
plt.xlabel('Month')
plt.ylabel('Industry')
plt.xticks(rotation=45)
plt.tight_layout()

# Save to PDF
plt.savefig('heatmap.pdf', format='pdf')
plt.show()

In [ ]:
# Create a PDF file to save the plots
pdf_pages = PdfPages('industry_stocks_volume_heatmaps.pdf')
months_order = ['OCT', 'SEP', 'AUG', 'JUL', 'JUN', 'MAY', 'APR', 'MAR', 'FEB', 'JAN', 'DEC', 'NOV']

industry_groups = monthly_return_volume.groupby('INDUSTRY')

for industry, group in industry_groups:
    industry_stocks = group[['SYMBOL', 'SECTOR', 'INDUSTRY', 'MARKET_CAP'] + months_order]
    industry_stocks_heatmap = industry_stocks.set_index('SYMBOL')
    industry_stocks_heatmap = industry_stocks_heatmap.drop(columns=['SECTOR', 'INDUSTRY', 'MARKET_CAP'])
    industry_stocks_heatmap = industry_stocks_heatmap.apply(pd.to_numeric, errors='coerce')
    
    # Plot heatmap
    plt.figure(figsize=(14, 3 + (3 * (group.shape[0] // 10)))) 
    sns.heatmap(industry_stocks_heatmap, annot=True, cmap="RdYlGn", fmt=".2f", vmin=-15, vmax=15)
    plt.title('Industry Stocks Returns Heatmap - ' + industry)
    plt.xlabel('Months')
    plt.ylabel('Stocks')
    
    # Save the plot to the PDF file
    pdf_pages.savefig()
    plt.close()

# Close the PDF file
pdf_pages.close()

### FNO Discount or Premium

In [ ]:
%%capture
fno_stocks = [symbol for symbol in fno_symbols if symbol not in ['NIFTY', 'NIFTYIT', 'BANKNIFTY']]
data = []
for stock in fno_stocks:
    try:
        ltp_future = nse_quote_ltp(stock, "next", "Fut")
        ltp_spot = nse_quote_ltp(stock)
        data.append({'SYMBOL': stock, 'LTP': ltp_spot, 'FUTURE_PRICE': ltp_future})
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")
data = pd.DataFrame(data)

In [ ]:
data['DIFF'] = round((data['FUTURE_PRICE'] - data['LTP']) * 100 / data['LTP'])
data.sort_values('DIFF').iloc[0:20]
#data.sort_values('DIFF', ascending=False).iloc[0:20]

In [ ]:
# oi_data, ltp, crontime = oi_chain_builder("INDIAMART","30-May-2024","full")

In [ ]:
# selected_columns = oi_data[['CALLS_OI', 'CALLS_LTP', 'CALLS_Bid Qty', 'CALLS_Bid Price', 'CALLS_Ask Price', 'CALLS_Ask Qty', 
#                             'Strike Price', 
#                             'PUTS_Bid Qty', 'PUTS_Bid Price', 'PUTS_Ask Price', 'PUTS_Ask Qty','PUTS_LTP', 'PUTS_OI']]
# selected_columns = selected_columns[(selected_columns['CALLS_OI'] > 0) | (selected_columns['PUTS_OI'] > 0)]
# selected_columns

### LIS Based Trading Strategy

In [ ]:
output_data = {'SYMBOL': [], 'LIS': []}

for symbol in fno_stocks:
    data = yf.download(tickers=symbol+".NS", period="5d", interval="1h", progress=False)
    close_prices = data['Close'].values
    lis = [1] * len(close_prices)
    for i in range(1,len(close_prices)):
        subproblems = [lis[k] for k in range(i) if close_prices[k] < close_prices[i]]
        lis[i] = 1 + max(subproblems, default=0)
    lis_max = max(lis, default=0)
    output_data['SYMBOL'].append(symbol)
    output_data['LIS'].append(lis_max)

output_df = pd.DataFrame(output_data)
output_df.set_index('SYMBOL', inplace=True)

In [ ]:
def download_stock_data(symbol):
    return yf.download(symbol + ".NS", start=pd.to_datetime('today') - pd.DateOffset(365),
                       end=pd.to_datetime('today') + pd.DateOffset(1), progress=False)

def calculate_technical_indicators(data):
    data['5DMA'] = data['Close'].rolling(window=5).mean()
    data['20DMA'] = data['Close'].rolling(window=20).mean()
    data['50DMA'] = data['Close'].rolling(window=50).mean()
    data['100DMA'] = data['Close'].rolling(window=100).mean()
    data['diff'] = data['Close'] - data['Open']
    data['color'] = data['diff'].apply(lambda x: 'green' if x >= 0 else 'red')
    return data

def plot_stock_data(symbol, data):
    # Prepare plot data
    plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=180))]

    # Setup subplot layout
    figure = make_subplots(specs=[[{"secondary_y": True}]])

    # Plot candlestick chart
    figure.add_trace(go.Candlestick(x=plot_data.index,
                                    open=plot_data['Open'],
                                    high=plot_data['High'],
                                    low=plot_data['Low'],
                                    close=plot_data['Close'],
                                    name='Price'))

    # Adjust Y-axis and layout for stock prices
    figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
    figure.update_xaxes(rangebreaks=[dict(bounds=['sat', 'mon'])], row=1, col=1)  # hide weekends
    figure.update_layout(title={'text': symbol, 'x': 0.5}, xaxis_rangeslider_visible=False)

    # Plot moving averages and volume
    for ma, color in zip(['5DMA', '20DMA', '50DMA', '100DMA'], ['yellow', 'blue', 'orange', 'green']):
        figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data[ma], marker_color=color, name=f'{ma}'))
    figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume',
                            marker={'color': plot_data['color']}), secondary_y=True)
    

    # Hide the secondary Y-axis (volume)
    figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
    figure.update_yaxes(visible=False, secondary_y=True)
    return figure


def process_stocks(all_symbols):
    num_processed = 0
    file_count = 1
    while num_processed < len(all_symbols):
        figure_html = open(f'last_5_days_buying_{file_count}.html', 'w')
        for symbol in all_symbols[num_processed:num_processed+100]:
            try:
                data = download_stock_data(symbol)
                data = calculate_technical_indicators(data)
                figure = plot_stock_data(symbol, data)
                figure_html.write(figure.to_html(full_html=False))
            except Exception as e:
                print(f"Error processing symbol {symbol}: {e}")
        figure_html.close()
        num_processed += 100
        file_count += 1


if __name__ == "__main__":
    symbols = output_df.sort_values('LIS', ascending=False).iloc[0:20].index
    process_stocks(symbols)
    print('Execution Completed!!!!')

In [ ]:
output_df.sort_values('LIS', ascending=False)[:50]